# Simulation Basics

These are the basic components we will use for our simulations. This notebook goes over how they work.

In [1]:
import numpy as np

from auctionbets.events import Event
from auctionbets.agents import Agent

## Agents

Agents represent people who make or lay bets. An agent just has their capital, which is the total stake of money they have left, and a list of ids for their open bets (bets they have not yet collected for). 

In [2]:
agent_1 = Agent(starting_capital=100.0)
agent_2 = Agent(starting_capital=100.0)

print(agent_1)
print(agent_2)

**Agent 0**
  Capital: 100.00
  Open bets:
**Agent 1**
  Capital: 100.00
  Open bets:


## Events

Events represent specific sporting events with a binary outcome (win or lose). These are the events that agents can bet on. 

Each event has a "true probability" of happening. This is unknown to the agents. 

In [3]:
event_1 = Event(true_probability=0.5)
print(event_1)

**Event 2**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:


Now agents can either bet or lay an event. 

If they bet the event, then they are betting the outcome happens. If it happens, they get their stake back, plus the odds * their stake. 

If the lay the event, they are taking the opposite side of that bet. Therefore, they lose if the outcome happens. If the outcome happens, they lose their money.

Here agent 1 bets on the event, and agent 2 lays it.

In [4]:
bet_1 = agent_1.bet(min_odds=1.5, stake=10.0, event=event_1)
bet_2 = agent_2.lay(max_odds=2.0, stake=30.0, event=event_1)

print(bet_1)
print(bet_2)

Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2)
Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2)


Now the event and the agents both have those bets associated with them:

In [5]:
print(agent_1)
print(agent_2)
print(event_1)

**Agent 0**
  Capital: 90.00
  Open bets:
    Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2)
**Agent 1**
  Capital: 70.00
  Open bets:
    Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2)
**Event 2**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:
    Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2) (unmatched)
    Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2) (unmatched)


## Running an event

An event needs to first be matched, which is where we run the double auction mechanism and match up all possible bets. Those bets that cannot be matched get their stake returned.

In [6]:
event = Event(true_probability=0.5)

n_agents = 20
agents_pool = [Agent(starting_capital=0) for _ in range(n_agents)]

is_lay_lst = np.random.choice([True, False], n_agents)
stake_lst = np.random.rand(n_agents) * (20.0 - 10.0) + 10.0
odds_lst = np.random.rand(n_agents) + 0.5

for agent, is_lay, stake, odds in zip(agents_pool, is_lay_lst, stake_lst, odds_lst):
    if is_lay:
        agent.lay(event=event, max_odds=odds + 0.1, stake=stake)
    else:
        agent.bet(event=event, min_odds=odds - 0.1, stake=stake)

print(event)

**Event 3**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:
    Bet(limit_odds=0.9589482171556404, stake=10.121330157712098, agent_idx=5, event_idx=3) (unmatched)
    Bet(limit_odds=1.1021913575105593, stake=13.953245977804585, agent_idx=6, event_idx=3) (unmatched)
    Bet(limit_odds=0.9309395503090993, stake=10.969928514259815, agent_idx=8, event_idx=3) (unmatched)
    Bet(limit_odds=1.0062928269129183, stake=18.276968409775435, agent_idx=9, event_idx=3) (unmatched)
    Bet(limit_odds=1.276340366127901, stake=13.677591909940771, agent_idx=11, event_idx=3) (unmatched)
    Bet(limit_odds=1.1210272911574104, stake=12.831402020378162, agent_idx=13, event_idx=3) (unmatched)
    Bet(limit_odds=0.9894588871246991, stake=11.396531097069392, agent_idx=16, event_idx=3) (unmatched)
    Bet(limit_odds=0.5414344592281355, stake=16.783826108285865, agent_idx=20, event_idx=3) (unmatched)
    Bet(limit_odds=0.7103427077282701, stake=12.317451841489731, agent_idx=21, event

In [7]:
event.match_bets()


New round
total_stake_bet = 0.0
total_stake_layed = 0.0
current_bet_odds = 0.5414344592281355
current_lay_odds = 1.359272947733516
Lay Needed
next_lay_odds = 1.2994079940244945
Lay matched

New round
total_stake_bet = 0.0
total_stake_layed = 12.375825722149383
current_bet_odds = 0.5414344592281355
current_lay_odds = 1.2994079940244945
Bet Needed
Bet matched

New round
total_stake_bet = 16.783826108285865
total_stake_layed = 12.375825722149383
current_bet_odds = 0.7103427077282701
current_lay_odds = 1.2994079940244945
Lay Needed
next_lay_odds = 1.2760704338610713
Lay matched

New round
total_stake_bet = 16.783826108285865
total_stake_layed = 32.09100677528129
current_bet_odds = 0.7103427077282701
current_lay_odds = 1.2760704338610713
Bet Needed
Bet matched

New round
total_stake_bet = 29.101277949775596
total_stake_layed = 32.09100677528129
current_bet_odds = 0.9309395503090993
current_lay_odds = 1.2760704338610713
Lay Needed
next_lay_odds = 1.2437152257620023
Lay matched

New round
to

In [8]:
print(event)

**Event 3**
  True Probability: 0.5
  Bettor Odds: 0.9894588871246991
  Layer Odds: 1.0908956165718169
  All bets:
    Bet(limit_odds=0.5414344592281355, stake=16.783826108285865, agent_idx=20, event_idx=3) (matched)
    Bet(limit_odds=0.7103427077282701, stake=12.317451841489731, agent_idx=21, event_idx=3) (matched)
    Bet(limit_odds=0.9309395503090993, stake=10.969928514259815, agent_idx=8, event_idx=3) (matched)
    Bet(limit_odds=0.9589482171556404, stake=10.121330157712098, agent_idx=5, event_idx=3) (matched)
    Bet(limit_odds=0.9755205449094754, stake=14.2145158579491, agent_idx=22, event_idx=3) (matched)
    Bet(limit_odds=0.9894588871246991, stake=11.396531097069392, agent_idx=16, event_idx=3) (unmatched)
    Bet(limit_odds=1.0062928269129183, stake=18.276968409775435, agent_idx=9, event_idx=3) (unmatched)
    Bet(limit_odds=1.1021913575105593, stake=13.953245977804585, agent_idx=6, event_idx=3) (unmatched)
    Bet(limit_odds=1.1210272911574104, stake=12.831402020378162, agen